# 9 - Metaprogramming

## Putting a Wrapper Around a Function

In [1]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper


In [2]:
@timethis
def countdown(n):
    while n > 0:
        n -= 1


In [5]:
countdown(1e6)

countdown 0.07303404808044434


## Preserving Function Metadata When Writing Decorators
Remember to use @wraps...

In [8]:
def action(func):
    def wrapper(*args, **kwargs):
        ''' Docs for wrapper '''
        result = func(*args, **kwargs)
        return result
    return wrapper

@action
def adder(a, b):
    ''' Docs for adder '''
    return a + b

adder(1, 2)

3

In [11]:
adder.__name__

'wrapper'

In [13]:
adder.__doc__

' Docs for wrapper '

In [14]:
adder.__annotations__

{}

If we use @wraps instead...

In [1]:
from functools import wraps

def action(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        ''' Docs for wrapper '''
        result = func(*args, **kwargs)
        return result
    return wrapper

@action
def adder(a, b):
    ''' Docs for adder '''
    return a + b

adder(1, 2)

3

In [17]:
adder.__name__

'adder'

In [18]:
adder.__doc__

' Docs for adder '

In [19]:
adder.__annotations__

{}

## Unwrapping a Decorator
If you have previously used @wraps

In [5]:
from functools import wraps

def offbyone(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        ''' Docs for wrapper '''
        result = func(*args, **kwargs) + 1
        return result
    return wrapper

@offbyone
def adder(a, b):
    ''' Docs for adder '''
    return a + b


adder(1, 2)

4

In [6]:
adder

<function __main__.adder(a, b)>

In [7]:
orig_add = adder.__wrapped__
orig_add(1, 2)

3

## Defining a Decorator That Takes Arguments

In [8]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
 
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
    
        return wrapper
    
    return decorate


In [9]:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

add(1,2)

3

## Defining a Decorator with User Adjustable Attributes

In [10]:
from functools import wraps, partial
import logging


# utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func


In [12]:
def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
 
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

         # attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper
    return decorate


In [13]:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

In [14]:
import logging

logging.basicConfig(level=logging.DEBUG)
add(2,3)

DEBUG:__main__:add


5

In [15]:
add.set_message('Add called')
add(2,3)

DEBUG:__main__:Add called


5

In [16]:
add.set_level(logging.WARNING)
add(2,3)

5

## Enforcing Type Checking on a Function Using a Decorator

In [17]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func
 
        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
 
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError('Argument {} must be {}'.format(name, bound_types[name]))
            return func(*args, **kwargs)
 
        return wrapper
    return decorate


In [18]:
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)


In [19]:
spam(1,2,3)

1 2 3


In [20]:
spam(1, 'hello', 3)

1 hello 3


In [21]:
spam(1, 'hello', 'world')

TypeError: Argument z must be <class 'int'>

In [22]:
__debug__

True

In [25]:
import inspect

sig = inspect.signature(spam)
sig

<Signature (x, y, z=42)>

In [26]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [27]:
sig.parameters['z'].name

'z'

In [28]:
sig.parameters['z'].default

42

In [29]:
sig.parameters['z'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [31]:
bound_types = sig.bind_partial(int,z=int)

In [32]:
bound_types

<BoundArguments (x=<class 'int'>, z=<class 'int'>)>

In [33]:
bound_types.arguments  # the missing arguments are simply ignored

OrderedDict([('x', int), ('z', int)])

## Defining Decorators As Part of a Class

In [35]:
from functools import wraps

class A:
    # decorator as an instance method
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper
 
    # decorator as a class method
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            return func(*args, **kwargs)
        return wrapper


In [36]:
a = A()
@a.decorator1  # applied from the instance a
def spam():
    pass


@A.decorator2  # applied from the class A
def grok():
    pass


In [37]:
spam()

Decorator 1


In [38]:
grok()

Decorator 2


## Defining Decorators As Classes
To define a decorator as an instance, you need to make sure it implements the __call__() and __get__() methods. 

In [39]:
import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0
 
    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)
 
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)


To use this class, you use it like a normal decorator, either inside or outside of a class.

In [40]:
@Profiled
def add(x, y):
    return x + y

class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)


In [41]:
add(1,2)

3

In [42]:
s = Spam()
s.bar(1)

<__main__.Spam object at 0x00000204A6315438> 1


In [43]:
add.ncalls

1

In [44]:
add(2,3)
add.ncalls

2

In [45]:
Spam.bar.ncalls

1

In [46]:
s.bar(2)
Spam.bar.ncalls

<__main__.Spam object at 0x00000204A6315438> 2


2

In [48]:
vars(Profiled)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Profiled.__init__(self, func)>,
              '__call__': <function __main__.Profiled.__call__(self, *args, **kwargs)>,
              '__get__': <function __main__.Profiled.__get__(self, instance, cls)>,
              '__dict__': <attribute '__dict__' of 'Profiled' objects>,
              '__weakref__': <attribute '__weakref__' of 'Profiled' objects>,
              '__doc__': None})

In [54]:
class A:
    pass

x = A()
y = A()

In [55]:
A.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [57]:
A.__weakref__.__doc__

'list of weak references to the object (if defined)'

In [60]:
#dir(A.__weakref__)

## Applying Decorators to Class and Static Methods
Make sure that your decorators are applied after @classmethod or @staticmethod.

In [1]:
import time
from functools import wraps

# a simple decorator
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        end = time.time()
        print(end-start)
        return r
    return wrapper


In [2]:
# Class illustrating application of the decorator to different kinds of methods
class Spam:
    @timethis
    def instance_method(self, n):
        print(self, n)
        while n > 0:
            n -= 1

    @classmethod
    @timethis
    def class_method(cls, n):
        print(cls, n)
        while n > 0:
            n -= 1

    @staticmethod
    @timethis
    def static_method(n):
        print(n)
        while n > 0:
            n -= 1


In [6]:
s = Spam()
s.instance_method(1e6)    

<__main__.Spam object at 0x000001AC37A79FD0> 1000000.0
0.10300064086914062


In [7]:
s.class_method(1e6) 

<class '__main__.Spam'> 1000000.0
0.10799980163574219


In [8]:
s = Spam()
s.static_method(1e6) 

1000000.0
0.06400036811828613


## Writing Decorators That Add Arguments to Wrapped Functions

Extra arguments can be injected into the calling signature using keyword-only arguments.

In [10]:
from functools import wraps

def optional_debug(func):
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)
    return wrapper


In [11]:
@optional_debug
def spam(a,b,c):
    print(a,b,c)


In [12]:
spam(1,2,3)

1 2 3


In [13]:
spam(1,2,3, debug=True)

Calling spam
1 2 3


if the @optional_debug decorator was applied to a function that already had a debug argument, then it would break. If that’s a concern, then we can add extra checking for it.

## Using Decorators to Patch Class Definitions
Class decorators can often be used as a straightforward alternative to other more advanced techniques involving mixins or metaclasses.


In [14]:
def log_getattribute(cls):
    # Get the original implementation
    orig_getattribute = cls.__getattribute__
 
    # Make a new definition
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)
 
    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls


In [15]:
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x
 
    def spam(self):
        pass


In [16]:
a = A(42)

In [17]:
a.x

getting: x


42

In [18]:
a.spam()

getting: spam


## Using a Metaclass to Control Instance Creation

In [19]:
class NoInstances(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("Can't instantiate directly")


In [20]:
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')


In [22]:
Spam.grok(1)

Spam.grok


In [23]:
s = Spam()

TypeError: Can't instantiate directly

Now, suppose you want to implement the singleton pattern.

In [1]:
class Singleton(type):
    def __init__(self, *args, **kwargs):
        self.__instance = None
        super().__init__(*args, **kwargs)
 
    def __call__(self, *args, **kwargs):
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs)
            return self.__instance
        else:
            return self.__instance


In [2]:
class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')


In [3]:
a = Spam()

Creating Spam


In [4]:
b = Spam()

In [5]:
a is b

True

Or suppose you want to create cached instances.

In [7]:
# basically caching, but also see flyweight pattern.
import weakref

class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()
 
    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj


In [8]:
class Spam(metaclass=Cached):
    def __init__(self, name):
        print('Creating Spam({!r})'.format(name))
        self.name = name


In [9]:
a = Spam('Guido')

Creating Spam('Guido')


In [10]:
b = Spam('Diana')

Creating Spam('Diana')


In [11]:
c = Spam('Guido')  # cached

In [13]:
a is c

True

In [14]:
a is b

False

## Capturing Class Attribute Definition Order
You want to automatically record the order in which attributes and methods are defined inside a class body so that you can use it in various operations.